In [1]:
import numpy as np
import random
from random import randint
from gurobipy import *
import pandas as pd
from random import seed

In [2]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2

In [3]:
# Product Demand
demand = np.random.randint(0,50,(Products,Market))

In [4]:
# Cost of opening
f_i = [20, 25]
f_j = [15, 5, 10]

In [5]:
# Unit cost of manufacturing product 
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

In [6]:
# Unit cost of transporting m from plant to DC
Transportation_i_j = np.random.uniform(0,5, (Products, Manufacturing_plants, Distribution))

In [7]:
# Unit cost of transporting m from DC to Market Zone
Transportation_j_k = np.random.uniform(0,5, (Products, Distribution, Market))

In [8]:
# Plant Capacities
Capacities_i = np.random.randint(300,500,(Manufacturing_plants)) # in volume !!! (metres cubed)
Capacities_j = np.random.randint(300,500,(Distribution)) # in volume !!! (metres cubed)
Capacities_l = np.zeros((Products,Outsourced)) # in terms of products 
np.fill_diagonal(Capacities_l, random.randint(15,20))

In [9]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
Supplier_cost = np.zeros((Products, Outsourced))
np.fill_diagonal(Supplier_cost, random.uniform(10,15))

In [10]:
# Cost of transporting product m from outsourced facility l to j
T_O_DC = np.zeros((Products, Outsourced, Distribution))
T_O_DC[0][0] = np.random.uniform(10,15,(Distribution))
T_O_DC[1][1] = np.random.uniform(10,15,(Distribution))

In [11]:
# Cost of shipping product m from outsourced facility l to k
T_O_MZ = np.zeros((Products, Outsourced, Market))
T_O_MZ[0][0] = np.random.uniform(15,20,(Market))
T_O_MZ[1][1] = np.random.uniform(15,20,(Market))

In [12]:
# Product volume 
volume = np.random.randint(2,5,(Products))

In [13]:
# unit cost of lost sales 
lost_sales = np.random.randint(50,100,(Market,Products))

In [14]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k

# Dictionaries for analysis 
Cost_dict = {}
Summary_dict = {} 

In [15]:
# Scenario parameters
a_i = np.ones(Manufacturing_plants)
b_j = np.ones(Distribution)

In [16]:
## Model

In [17]:
grbModel = Model('synthetic')

Using license file C:\Users\Devika Kabe\gurobi.lic
Academic license - for non-commercial use only


In [18]:
def SetGurobiModel():
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel.addVar(vtype = GRB.BINARY)
    
    for k in range(Market):
        for m in range(Products):
            U_km[k,m] = grbModel.addVar(vtype = GRB.INTEGER)
    
    for m in range(Products):
        for l in range(Outsourced):
            V_lm[m,l] = grbModel.addVar(vtype = GRB.INTEGER)
            
    for m in range(Products):
        for i in range(Manufacturing_plants):
            Q_im[m,i] = grbModel.addVar(vtype = GRB.INTEGER)
    
    for m in range(Products):
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                Y_ijm[m,i,j] = grbModel.addVar(vtype = GRB.INTEGER)
                
    for m in range(Products):
        for j in range(Distribution):
            for k in range(Market): 
                Z_jkm[m,j,k] = grbModel.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            for j in range(Distribution):
                T_ljm[m,l,j] = grbModel.addVar(vtype = GRB.INTEGER)      
                
    for m in range(Products):
        for l in range(Outsourced):
            for k in range(Market):
                T_lkm[m,l,k] = grbModel.addVar(vtype = GRB.INTEGER)
                
    SetGrb_Obj()
    ModelCons()

In [19]:
def SolveModel():
    
    grbModel.params.OutputFlag = 0
    grbModel.optimize()
    
    # get variable values 
    v_val_x_i = grbModel.getAttr('x', x_i)
    v_val_x_j = grbModel.getAttr('x', x_j)
    v_val_U_km = grbModel.getAttr('x', U_km)
    v_val_V_lm = grbModel.getAttr('x', V_lm)
    v_val_Q_im = grbModel.getAttr('x', Q_im)
    v_val_Y_ijm = grbModel.getAttr('x', Y_ijm)
    v_val_Z_jkm = grbModel.getAttr('x', Z_jkm)
    v_val_T_ljm = grbModel.getAttr('x', T_ljm)
    v_val_T_lkm = grbModel.getAttr('x', T_lkm)
    
    obj = grbModel.getObjective()
    print("obj val: ", obj.getValue())   
    
    Summary_dict["OpenMPs"] = np.sum(v_val_x_i.values())
    Summary_dict["OpenDCs"] = np.sum(v_val_x_j.values())
    Summary_dict["Purchasing"] = np.sum(v_val_V_lm.values())
    Summary_dict["Production"] = np.sum(v_val_Q_im.values())
    Summary_dict["LostSales"] = np.sum(v_val_U_km.values())
    Summary_dict["OutsourceToDC"] =  np.sum(v_val_T_ljm.values())
    Summary_dict["OutsourceToMarket"] = np.sum(v_val_T_lkm.values())
    
    Cost_dict["Opening"] =  get_opening_costs(v_val_x_i, v_val_x_j)
    Cost_dict["InHouseShipping"] = get_shipping_costs(v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[0]
    Cost_dict["OutsourceShipping"] = get_shipping_costs(v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[1]
    Cost_dict["Production"] = get_production_cost(v_val_Q_im)
    Cost_dict["Purchasing"] = get_purchase_costs(v_val_V_lm)
    Cost_dict["LostSales"] = get_lost_cost(v_val_U_km)    
    
    return

In [20]:
a_i = np.ones(Manufacturing_plants)
b_j = np.ones(Distribution)

In [21]:
# Objective

def SetGrb_Obj():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]

    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0

    # Shipment
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[m][l]*V_lm[m,l]

    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U_km[k,m]

    grb_expr += OC_1 + OC_2 + ship_1 + ship_2 + ship_3 + ship_4 + pr_cost + b_cost + l_cost
    
    grbModel.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 


In [22]:
# Model Constraints

def ModelCons():
    
    # Network Flow

    grbModel.addConstrs(quicksum(Y_ijm[m,i,j] for j in range(Distribution)) == Q_im[m,i]
                        for i in range(Manufacturing_plants) for m in range(Products))

    grbModel.addConstrs(quicksum(Z_jkm[m,j,k] for k in range(Market)) ==
                        (quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[m,l,j] for l in range(Outsourced))) for j in range(Distribution)
                        for m in range(Products))

    grbModel.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[m,l,k] for l in range(Outsourced))) == (demand[m][k] - U_km[k,m])
                        for k in range(Market) for m in range(Products))
    
    # Can only get one product type from each outsourced facility

    for m in range(Products):
        for l in range(Outsourced):
            if l != m:
                grbModel.addConstr(V_lm[m,l] == 0)
            
    for m in range(Products):
        for l in range(Outsourced):
            for j in range(Distribution):
                if l != m:
                    grbModel.addConstr(T_ljm[m,l,j] == 0)
                    
    for m in range(Products):
        for l in range(Outsourced):
            for k in range(Market):
                if l != m:
                    grbModel.addConstr(T_lkm[m,l,k] == 0)
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel.addConstrs(V_lm[m,l] == quicksum(T_ljm[m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[m,l,k] for k in range(Market)) for m in range(Products) for l in range(Outsourced))
    
    
    # Capacity Constraints
    grbModel.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= a_i[i]*Capacities_i[i]*x_i[i] 
                        for i in range(Manufacturing_plants))
    grbModel.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        b_j[j]*Capacities_j[j]*x_j[j] for j in range(Distribution))
    grbModel.addConstrs(quicksum(V_lm[m,l] for m in range(Products)) <= np.sum(Capacities_l[l]) for l in range(Outsourced))
    
    return   


In [24]:
def get_opening_costs(x1, x2):
    
    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x1[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x2[j]

    Opening = np.round(OC_1 + OC_2)
    
    return(Opening)
   
def get_shipping_costs(Y, Z, T1, T2):
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0

    # Shipment
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T1[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T2[m,l,k]
    
    in_house_shipping = np.round(ship_1 + ship_2)

    outsourced_shipping = np.round(ship_3 + ship_4)
    
    return(in_house_shipping, outsourced_shipping)

def get_production_cost(Q):

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += np.round(Manufacturing_costs[i][m]*Q[m,i])
            
    return(pr_cost)

def get_purchase_costs(V):    

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += np.round(Supplier_cost[m][l]*V[m,l])
            
    return(b_cost)

def get_lost_cost(U):
    
    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += np.round(lost_sales[k][m]*U[k,m])
            
    return(l_cost) 

In [25]:
def run_Model(MP_param, DC_param):
    for i in range(len(MP_param)):
        a_i[i] = MP_param[i]
    for j in range(len(DC_param)):
        b_j[j] = DC_param[j]
        
    SetGurobiModel()
    SolveModel()

In [49]:
def dict_to_dataframe(Dict):
    return pd.DataFrame([list(Dict.values())], columns = list(Dict.keys()))

In [53]:
run_Model([1,1],[1,1,1]) # No disrupted facilities

obj val:  876.446674459141


In [54]:
dict_to_dataframe(Summary_dict)

,OpenMPs,OpenDCs,Purchasing,Production,LostSales,OutsourceToDC,OutsourceToMarket
0,2.0,3.0,3.0,192.0,0.0,0.0,3.0


In [55]:
dict_to_dataframe(Cost_dict)

,Opening,InHouseShipping,OutsourceShipping,Production,Purchasing,LostSales
0,75.0,467.0,47.0,255.0,32.0,0.0


In [60]:
run_Model([0,0],[0,0,0]) # All disrupted facilities

obj val:  43011.01498493838


In [61]:
dict_to_dataframe(Summary_dict)

,OpenMPs,OpenDCs,Purchasing,Production,LostSales,OutsourceToDC,OutsourceToMarket
0,0.0,0.0,40.0,0.0,155.0,0.0,40.0


In [62]:
dict_to_dataframe(Cost_dict)

,Opening,InHouseShipping,OutsourceShipping,Production,Purchasing,LostSales
0,0.0,0.0,682.0,0.0,432.0,41896.0


In [63]:
run_Model([0,1],[1,0,1]) # 1 disrupted plant and 1 disrupted DC

obj val:  11520.362335912314


In [64]:
dict_to_dataframe(Summary_dict)

,OpenMPs,OpenDCs,Purchasing,Production,LostSales,OutsourceToDC,OutsourceToMarket
0,1.0,2.0,40.0,115.0,40.0,33.0,7.0


In [65]:
dict_to_dataframe(Cost_dict)

,Opening,InHouseShipping,OutsourceShipping,Production,Purchasing,LostSales
0,50.0,359.0,508.0,178.0,432.0,9992.0
